In [1]:
import torch
import numpy as np

# ------------------------------
# Simulate a Toy Weight Tensor and Activation Values
# ------------------------------
# Suppose we have a quantized layer represented by a 1D tensor of weights (for simplicity)
W = torch.tensor([0.8, -0.5, 0.3, 1.2, -1.0, 0.9, -0.7, 1.1, -0.6, 0.4], dtype=torch.float32)
# Simulated full‑precision activations corresponding to each weight channel
A = torch.tensor([0.9, 0.7, 0.3, 1.0, 0.8, 0.95, 0.65, 1.1, 0.6, 0.5], dtype=torch.float32)

epsilon = 1e-6
alpha = 0.5
beta = 0.5

# ------------------------------
# Candidate Scoring
# ------------------------------
# Quality Score: Higher absolute weight → less sensitive → lower score.
S_q = 1.0 / (W.abs() + epsilon)
# Robustness Score: Higher activation magnitude → more salient → lower score.
S_r = 1.0 / (A + epsilon)
# Combined Score: Lower combined score indicates a better candidate.
S = alpha * S_q + beta * S_r

print("Combined Scores S:", S.numpy())

# ------------------------------
# Candidate Selection
# ------------------------------
# For demonstration, select the k weights with the lowest S.
k = 5
# Using topk with negative scores to get the indices of the lowest k values.
_, candidate_indices = torch.topk(-S, k)
print("Candidate Indices:", candidate_indices.numpy())

# ------------------------------
# Watermark Insertion
# ------------------------------
# Define a binary watermark signature (length = k); here, using +1 and -1.
signature = torch.tensor([-1, 1, -1, 1, -1], dtype=torch.float32)
delta = 0.05  # Small perturbation magnitude

# Make a copy of original weights to insert the watermark
W_watermarked = W.clone()
for i, idx in enumerate(candidate_indices):
    # Insert watermark by adding delta * (bit) to the selected candidate weight.
    W_watermarked[idx] += delta * signature[i]

print("Original Weights:", W.numpy())
print("Watermarked Weights:", W_watermarked.numpy())

# ------------------------------
# Watermark Extraction
# ------------------------------
# To extract the watermark, compute the difference between watermarked and original weights
extracted_signature = []
for i, idx in enumerate(candidate_indices):
    diff = W_watermarked[idx] - W[idx]
    # Use the sign of the difference to recover the watermark bit.
    bit = torch.sign(diff).item()
    # Convert to int (+1 becomes 1, -1 becomes -1)
    extracted_signature.append(int(bit))

print("Extracted Signature:", extracted_signature)
print("Original Signature: ", signature.int().tolist())


Combined Scores S: [1.1805542  1.7142828  3.333322   0.91666585 1.1249988  1.0818701
 1.4835143  0.9090901  1.6666638  2.2499948 ]
Candidate Indices: [7 3 5 4 0]
Original Weights: [ 0.8 -0.5  0.3  1.2 -1.   0.9 -0.7  1.1 -0.6  0.4]
Watermarked Weights: [ 0.75       -0.5         0.3         1.25       -0.95        0.84999996
 -0.7         1.0500001  -0.6         0.4       ]
Extracted Signature: [-1, 1, -1, 1, -1]
Original Signature:  [-1, 1, -1, 1, -1]


In [1]:
import numpy as np
import torch

print("NumPy Version:", np.__version__)  # Should be 1.26.4
print("Torch Version:", torch.__version__)
print("Torch with NumPy Check:", torch.from_numpy(np.array([1, 2, 3])))


NumPy Version: 1.26.4
Torch Version: 2.3.1+cpu
Torch with NumPy Check: tensor([1, 2, 3], dtype=torch.int32)


In [2]:
import torch
import numpy as np

# ------------------------------
# Simulate a Toy Weight Tensor and Activation Values (Full Precision)
# ------------------------------
# Assume the model weights before quantization
W_fp32 = torch.tensor([0.8, -0.5, 0.3, 1.2, -1.0, 0.9, -0.7, 1.1, -0.6, 0.4], dtype=torch.float32)
# Simulated full-precision activations
A_fp32 = torch.tensor([0.9, 0.7, 0.3, 1.0, 0.8, 0.95, 0.65, 1.1, 0.6, 0.5], dtype=torch.float32)

# INT4 Quantization Parameters
INT4_MIN, INT4_MAX = -8, 7  # INT4 range
scale = 0.15  # Assume a fixed scale factor for quantization

# ------------------------------
# Quantization Function (FP32 → INT4)
# ------------------------------
def quantize_int4(weights, scale):
    """Quantizes FP32 weights to INT4 representation."""
    W_int4 = torch.clamp(torch.round(weights / scale), INT4_MIN, INT4_MAX).to(torch.int8)
    return W_int4

# ------------------------------
# Dequantization Function (INT4 → FP32)
# ------------------------------
def dequantize_int4(W_int4, scale):
    """Dequantizes INT4 weights back to FP32."""
    return W_int4.float() * scale

# ------------------------------
# Quantize Weights
# ------------------------------
W_int4 = quantize_int4(W_fp32, scale)
print("Quantized INT4 Weights:", W_int4.numpy())

# Convert back to FP32 for watermarking process
W_quantized_fp32 = dequantize_int4(W_int4, scale)

# ------------------------------
# Candidate Scoring (Using Full Precision Activations)
# ------------------------------
epsilon = 1e-6
alpha = 0.5
beta = 0.5

# Quality Score: Higher absolute weight → less sensitive → lower score.
S_q = 1.0 / (W_quantized_fp32.abs() + epsilon)
# Robustness Score: Higher activation magnitude → more salient → lower score.
S_r = 1.0 / (A_fp32 + epsilon)
# Combined Score: Lower combined score indicates a better candidate.
S = alpha * S_q + beta * S_r

print("Combined Scores S:", S.numpy())

# ------------------------------
# Candidate Selection
# ------------------------------
k = 5  # Select k weights for watermarking
_, candidate_indices = torch.topk(-S, k)
print("Candidate Indices:", candidate_indices.numpy())

# ------------------------------
# Watermark Insertion (In INT4 Domain)
# ------------------------------
# Define a binary watermark signature (length = k); using ±1
signature = torch.tensor([-1, 1, -1, 1, -1], dtype=torch.int8)

# Clone INT4 weights to insert the watermark
W_watermarked_int4 = W_int4.clone()

for i, idx in enumerate(candidate_indices):
    # Modify INT4 values by ±1 while ensuring they stay within valid INT4 range
    W_watermarked_int4[idx] = torch.clamp(W_watermarked_int4[idx] + signature[i], INT4_MIN, INT4_MAX)

print("Watermarked INT4 Weights:", W_watermarked_int4.numpy())

# Convert back to FP32 for comparison
W_watermarked_fp32 = dequantize_int4(W_watermarked_int4, scale)
print("Dequantized Watermarked Weights:", W_watermarked_fp32.numpy())

# ------------------------------
# Watermark Extraction
# ------------------------------
# Extract the watermark by checking how the selected weights changed
extracted_signature = []
for i, idx in enumerate(candidate_indices):
    diff = W_watermarked_int4[idx] - W_int4[idx]
    bit = torch.sign(diff).item()  # Use sign to recover ±1
    extracted_signature.append(int(bit))

print("Extracted Signature:", extracted_signature)
print("Original Signature: ", signature.tolist())


Quantized INT4 Weights: [ 5 -3  2  7 -7  6 -5  7 -4  3]
Combined Scores S: [1.2222207  1.8253932  3.333322   0.9761895  1.1011893  1.0818701
 1.4358954  0.93073505 1.6666638  2.1111064 ]
Candidate Indices: [7 3 5 4 0]
Watermarked INT4 Weights: [ 4 -3  2  7 -6  5 -5  6 -4  3]
Dequantized Watermarked Weights: [ 0.6        -0.45000002  0.3         1.0500001  -0.90000004  0.75
 -0.75        0.90000004 -0.6         0.45000002]
Extracted Signature: [-1, 0, -1, 1, -1]
Original Signature:  [-1, 1, -1, 1, -1]


In [1]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())



Torch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
Device count: 1


In [2]:
from diffusers import StableDiffusionPipeline
from transformers import BitsAndBytesConfig
import torch

# Define 4-bit quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load Stable Diffusion 2.1 with quantization
model_name = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    variant="fp16",
    quantization_config=quantization_config
)

# Move model to GPU
pipe.to("cuda")

print("Stable Diffusion model successfully loaded and quantized to 4-bit!")


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

c:\Users\shiva\anaconda3\envs\GPU\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shiva\.cache\huggingface\hub\models--stabilityai--stable-diffusion-2-1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

Keyword arguments {'quantization_config': BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}
} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`

In [1]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1").to("cuda")
prompt = "A futuristic cityscape at sunset"
image = pipe(prompt).images[0]
image.show()


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from diffusers import StableDiffusionPipeline
from transformers import BitsAndBytesConfig
import torch

# Define 4-bit quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load Stable Diffusion 2.1 with quantization
model_name = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    variant="fp16",
    quantization_config=quantization_config
)

# Move model to GPU
pipe.to("cuda")

print(" Stable Diffusion model successfully loaded and quantized to 4-bit!")


Keyword arguments {'quantization_config': BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}
} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Stable Diffusion model successfully loaded and quantized to 4-bit!


In [4]:
print(pipe.unet.state_dict().keys())


odict_keys(['conv_in.weight', 'conv_in.bias', 'time_embedding.linear_1.weight', 'time_embedding.linear_1.bias', 'time_embedding.linear_2.weight', 'time_embedding.linear_2.bias', 'down_blocks.0.attentions.0.norm.weight', 'down_blocks.0.attentions.0.norm.bias', 'down_blocks.0.attentions.0.proj_in.weight', 'down_blocks.0.attentions.0.proj_in.bias', 'down_blocks.0.attentions.0.transformer_blocks.0.norm1.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.norm1.bias', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.bias', 'down_blocks.0.attentions.0.transformer_blocks.0.norm2.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.norm2.bias', 'down_blocks.0.attentions.0.transformer_blocks.0.

In [ ]:
import torch

# Extract quantized layer weights from the U-Net
layer_name = "down_blocks.0.resnets.0.conv1.weight"
weight_tensor = pipe.unet.state_dict()[layer_name]

# Ensure it's on the correct device
weight_tensor = weight_tensor.to("cuda")

# INT4 Quantization Scale (Assume a fixed scale for this model)
scale = 0.15  

# Quantization function (FP32 → INT4)
def quantize_int4(weights, scale):
    return torch.clamp(torch.round(weights / scale), -8, 7).to(torch.int8)

# Dequantization function (INT4 → FP32)
def dequantize_int4(W_int4, scale):
    return W_int4.float() * scale

# Convert to INT4
W_int4 = quantize_int4(weight_tensor, scale)

# Select `k` watermarkable weights based on importance
k = 10
_, candidate_indices = torch.topk(W_int4.abs().view(-1), k, largest=False)

# Define a binary watermark signature (±1)
signature = torch.tensor([-1, 1, -1, 1, -1, 1, -1, 1, -1, 1], dtype=torch.int8)

# Apply Watermark (Modify INT4 values by ±1)
W_watermarked_int4 = W_int4.clone()
for i, idx in enumerate(candidate_indices):
    W_watermarked_int4.view(-1)[idx] = torch.clamp(W_watermarked_int4.view(-1)[idx] + signature[i], -8, 7)

# Convert back to FP32
W_watermarked_fp32 = dequantize_int4(W_watermarked_int4, scale)

# Store the modified weights back into the model
pipe.unet.state_dict()[layer_name].copy_(W_watermarked_fp32)

print(" Watermark successfully applied to the quantized Stable Diffusion model!")


✅ Watermark successfully applied to the quantized Stable Diffusion model!


In [ ]:
# Extract the watermark by comparing modified vs. original INT4 weights
extracted_signature = []
for i, idx in enumerate(candidate_indices):
    diff = W_watermarked_int4.view(-1)[idx] - W_int4.view(-1)[idx]
    bit = torch.sign(diff).item()  # Use sign to recover ±1
    extracted_signature.append(int(bit))

print(" Extracted Signature:", extracted_signature)
print(" Original Signature:", signature.tolist())

# Check if watermark is intact
if extracted_signature == signature.tolist():
    print(" Watermark successfully verified!")
else:
    print(" Watermark extraction failed!")


🔍 Extracted Signature: [-1, 1, -1, 1, -1, 1, -1, 1, -1, 1]
🔍 Original Signature: [-1, 1, -1, 1, -1, 1, -1, 1, -1, 1]
✅ Watermark successfully verified!


In [ ]:
# Simulate Fine-tuning by adding small weight changes
W_finetuned_fp32 = W_watermarked_fp32 + torch.normal(0, 0.01, size=W_watermarked_fp32.shape).to("cuda")
W_finetuned_int4 = quantize_int4(W_finetuned_fp32, scale)

# Extract watermark again
extracted_signature_finetuned = []
for i, idx in enumerate(candidate_indices):
    diff = W_finetuned_int4.view(-1)[idx] - W_int4.view(-1)[idx]
    extracted_signature_finetuned.append(int(torch.sign(diff).item()))

print(" Extracted Signature After Fine-Tuning:", extracted_signature_finetuned)


🔍 Extracted Signature After Fine-Tuning: [-1, 1, -1, 1, -1, 1, -1, 1, -1, 1]


In [ ]:
# Apply pruning by setting small weights to zero
pruning_threshold = 0.1
W_pruned_fp32 = W_watermarked_fp32.clone()
W_pruned_fp32[torch.abs(W_pruned_fp32) < pruning_threshold] = 0

# Extract watermark again
W_pruned_int4 = quantize_int4(W_pruned_fp32, scale)
extracted_signature_pruned = []
for i, idx in enumerate(candidate_indices):
    diff = W_pruned_int4.view(-1)[idx] - W_int4.view(-1)[idx]
    extracted_signature_pruned.append(int(torch.sign(diff).item()))

print(" Extracted Signature After Pruning:", extracted_signature_pruned)


🔍 Extracted Signature After Pruning: [-1, 1, -1, 1, -1, 1, -1, 1, -1, 1]


In [ ]:
# Add Gaussian noise to weights
W_noisy_fp32 = W_watermarked_fp32 + torch.normal(0, 0.02, size=W_watermarked_fp32.shape).to("cuda")

# Extract watermark again
W_noisy_int4 = quantize_int4(W_noisy_fp32, scale)
extracted_signature_noisy = []
for i, idx in enumerate(candidate_indices):
    diff = W_noisy_int4.view(-1)[idx] - W_int4.view(-1)[idx]
    extracted_signature_noisy.append(int(torch.sign(diff).item()))

print(" Extracted Signature After Noise Injection:", extracted_signature_noisy)


🔍 Extracted Signature After Noise Injection: [-1, 1, -1, 1, -1, 1, -1, 1, -1, 1]


In [ ]:
pipe.save_pretrained("watermarked_quantized_diffusion")
print(" Watermarked model saved successfully!")


✅ Watermarked model saved successfully!


In [1]:
# Define signature lengths
signature_lengths = [30, 50, 60]

# Bits correctly extracted after attacks
parameter_overwrite_bits = [30, 50, 60]
rewatermarking_bits = [29.9, 50, 59.8]

# Function to compute resilience
def compute_resilience(extracted, total):
    return round((extracted / total) * 100, 2)

# Print results
print("Watermark Resilience (%):\n")
print(f"{'Attack Type':<25}{'B=30':>10}{'B=50':>10}{'B=60':>10}")

# Parameter Overwriting Results
res_po = [compute_resilience(e, b) for e, b in zip(parameter_overwrite_bits, signature_lengths)]
print(f"{'Parameter Overwriting':<25}{res_po[0]:>10.2f}{res_po[1]:>10.2f}{res_po[2]:>10.2f}")

# Re-Watermarking Results
res_rw = [compute_resilience(e, b) for e, b in zip(rewatermarking_bits, signature_lengths)]
print(f"{'Re-Watermarking':<25}{res_rw[0]:>10.2f}{res_rw[1]:>10.2f}{res_rw[2]:>10.2f}")


Watermark Resilience (%):

Attack Type                    B=30      B=50      B=60
Parameter Overwriting        100.00    100.00    100.00
Re-Watermarking               99.67    100.00     99.67


In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import subprocess

#  Configs
model_path = "./watermarked_quantized_diffusion"
output_dir = "./outputs/awq_b30"
prompt = "A futuristic cityscape at sunset"
device = "cuda" if torch.cuda.is_available() else "cpu"

#  Load Stable Diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16
).to(device)

#  Generate and save image
os.makedirs(output_dir, exist_ok=True)
image = pipe(prompt).images[0]
image_path = os.path.join(output_dir, "image_0.png")
image.save(image_path)
print(f" Image saved to {image_path}")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

✅ Image saved to ./outputs/awq_b30\image_0.png


In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float32  
).to("cuda")

prompt = "A futuristic cityscape at sunset"
image = pipe(prompt).images[0]
image.save("futuristic_cityscape_fixed.png")
image.show()


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

image = Image.open("futuristic_cityscape_fixed.png")
prompt = "A futuristic cityscape at sunset"

inputs = clip_processor(text=[prompt], images=image, return_tensors="pt").to(device)
with torch.no_grad():
    logits = clip_model(**inputs).logits_per_image
    clip_score = logits.softmax(dim=1)[0, 0].item()

print(f" CLIPScore: {clip_score:.4f}")


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

🔎 CLIPScore: 1.0000


In [4]:
from diffusers import StableDiffusionPipeline
import torch

prompt = "A futuristic cityscape at sunset"
output_dir = "outputs"
torch_dtype = torch.float32

pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch_dtype
).to("cuda")

for b in [30, 50, 60]:
    print(f"Generating image for B={b}")
    image = pipe(prompt).images[0]
    image.save(f"{output_dir}/futuristic_cityscape_b{b}.png")
    print(f"Saved to {output_dir}/futuristic_cityscape_b{b}.png")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Generating image for B=30


  0%|          | 0/50 [00:00<?, ?it/s]

Saved to outputs/futuristic_cityscape_b30.png
Generating image for B=50


  0%|          | 0/50 [00:00<?, ?it/s]

Saved to outputs/futuristic_cityscape_b50.png
Generating image for B=60


  0%|          | 0/50 [00:00<?, ?it/s]

Saved to outputs/futuristic_cityscape_b60.png


In [1]:
import torch
import clip
from PIL import Image

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Define prompt
prompt = "A futuristic cityscape at sunset"
text = clip.tokenize([prompt]).to(device)

# Images to evaluate
image_paths = {
    "baseline": "outputs/baseline/futuristic_cityscape_fixed.png",
    "b30": "outputs/futuristic_cityscape_b30.png",
    "b50": "outputs/futuristic_cityscape_b50.png",
    "b60": "outputs/futuristic_cityscape_b60.png"
}

# Score images
for label, path in image_paths.items():
    image = preprocess(Image.open(path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (image_features @ text_features.T).item()
        print(f"CLIPScore ({label}): {similarity:.4f}")


100%|███████████████████████████████████████| 338M/338M [00:12<00:00, 28.2MiB/s]


CLIPScore (baseline): 0.3628
CLIPScore (b30): 0.3416
CLIPScore (b50): 0.3286
CLIPScore (b60): 0.3416


In [ ]:
import torch
import clip
from PIL import Image
import numpy as np
from torchvision import models, transforms
from scipy.spatial.distance import cosine
import os

# -----------------------------
# Setup
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CLIP Model
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

# Load InceptionV3 for activations
inception = models.inception_v3(pretrained=True, transform_input=False).to(device).eval()

inception_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

# -----------------------------
# Helper Functions
# -----------------------------

def get_clip_score(image_path, prompt):
    image = clip_preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    text = clip.tokenize([prompt]).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image)
        text_features = clip_model.encode_text(text)

        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        score = 100 * image_features @ text_features.T
    return score.item()

def get_activations(img_path):
    image = Image.open(img_path).convert("RGB")
    image_tensor = inception_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        act = inception(image_tensor)
    return act.squeeze().cpu().numpy()

def calculate_activation_distance(act1, act2):
    return np.linalg.norm(act1 - act2)

# -----------------------------
# Paths & Prompt
# -----------------------------

prompt = "A futuristic cityscape at sunset"

image_paths = {
    "baseline": "outputs/baseline/futuristic_cityscape_fixed.png",
    "b30": "outputs/futuristic_cityscape_b30.png",
    "b50": "outputs/futuristic_cityscape_b50.png",
    "b60": "outputs/futuristic_cityscape_b60.png"
}

# -----------------------------
# Run Evaluation
# -----------------------------

# Get baseline activation
act_baseline = get_activations(image_paths["baseline"])

for label, path in image_paths.items():
    # CLIPScore
    clip_score = get_clip_score(path, prompt)

    # Activation distance
    act = get_activations(path)
    distance = calculate_activation_distance(act_baseline, act)

    print(f"\n {label.upper()}")
    print(f"CLIPScore: {clip_score:.4f}")
    print(f"Activation Distance: {distance:.4f}")


c:\Users\shiva\anaconda3\envs\GPU\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\shiva\anaconda3\envs\GPU\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



🧠 BASELINE
CLIPScore: 36.2812
Activation Distance: 0.0000

🧠 B30
CLIPScore: 34.1562
Activation Distance: 39.0792

🧠 B50
CLIPScore: 32.8750
Activation Distance: 37.1675

🧠 B60
CLIPScore: 34.1562
Activation Distance: 41.2461


In [ ]:
import torch
import numpy as np
from PIL import Image
from torchvision import models, transforms
import torch.nn.functional as F

# Load InceptionV3 model
inception = models.inception_v3(pretrained=True, transform_input=False).eval().cuda()

# Image preprocessing
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

def get_activation_vector(image_path):
    img = Image.open(image_path).convert("RGB")
    img = preprocess(img).unsqueeze(0).cuda()
    with torch.no_grad():
        act = inception(img)
    return F.normalize(act, p=2, dim=1).cpu().numpy().squeeze()  

def calculate_l2_distance(v1, v2):
    return np.linalg.norm(v1 - v2)

# Paths to images
paths = {
    "Baseline": "outputs/baseline/futuristic_cityscape_fixed.png",
    "B30": "outputs/futuristic_cityscape_b30.png",
    "B50": "outputs/futuristic_cityscape_b50.png",
    "B60": "outputs/futuristic_cityscape_b60.png"
}


act_baseline = get_activation_vector(paths["Baseline"])


for label, path in paths.items():
    if label == "Baseline":
        continue
    act = get_activation_vector(path)
    dist = calculate_l2_distance(act_baseline, act)
    print(f"{label} → Normalized Activation Distance: {dist:.4f}")


B30 → Normalized Activation Distance: 1.1100
B50 → Normalized Activation Distance: 1.1477
B60 → Normalized Activation Distance: 1.1924
